## Exploring the CONLL 2003 dataset using Autolabel

#### Setup the API Keys for providers that you want to use

In [1]:
import os

# provide your own OpenAI API key here
os.environ["OPENAI_API_KEY"] = "sk-xxxxxxxxxxxxxxxxx"

#### Install the autolabel library

In [2]:
!pip install 'refuel-autolabel[openai]'

#### Download the dataset

In [1]:
from autolabel import get_data

get_data("conll2003")

This downloads two datasets:
* `test.csv`: This is the larger dataset we are trying to label using LLMs
* `seed.csv`: This is a small dataset where we already have human-provided labels

## Start the labeling process!

Labeling with Autolabel is a 3-step process:
* First, we specify a labeling configuration (see `config.json` below)
* Next, we do a dry-run on our dataset using the LLM specified in `config.json` by running `agent.plan`
* Finally, we run the labeling with `agent.run`

### First labeling run

In [12]:
import json

from autolabel import LabelingAgent

In [13]:
# load the config
with open("config_conll2003.json") as f:
     config = json.load(f)

Let's review the configuration file below. You'll notice the following useful keys:
* `task_type`: `named_entity_recognition` (since it's a named entity recognition task)
* `model`: `{'provider': 'openai', 'name': 'gpt-3.5-turbo'}` (use a specific OpenAI model)
* `prompt.task_guidelines`: `'You are an expert at extracting Person, Organization, Location, and Miscellaneous entities...` (how we describe the task to the LLM)
* `prompt.labels`: `[
            "Location",
            "Organization",
            "Person",
            "Miscellaneous"
        ]` (the full list of labels to choose from)
* `prompt.few_shot_num`: 3 (how many labeled examples to provide to the LLM)

In [15]:
config

{'task_name': 'PersonLocationOrgMiscNER',
 'task_type': 'named_entity_recognition',
 'dataset': {'label_column': 'CategorizedLabels',
  'text_column': 'example',
  'delimiter': ','},
 'model': {'provider': 'openai', 'name': 'gpt-3.5-turbo'},
 'prompt': {'task_guidelines': 'You are an expert at extracting Person, Organization, Location, and Miscellaneous entities from text. Your job is to extract named entities mentioned in text, and classify them into one of the following categories.\nCategories:\n{labels}\n ',
  'labels': ['Location', 'Organization', 'Person', 'Miscellaneous'],
  'example_template': 'Example: {example}\nOutput:\n{CategorizedLabels}',
  'few_shot_examples': 'seed.csv',
  'few_shot_selection': 'semantic_similarity',
  'few_shot_num': 5}}

In [16]:
# create an agent for labeling
agent = LabelingAgent(config=config)

In [17]:
# dry-run -- this tells us how much this will cost and shows an example prompt
from autolabel import AutolabelDataset

ds = AutolabelDataset("data/conll2003/test.csv", config=config)
agent.plan(ds)

/opt/homebrew/lib/python3.10/site-packages/rich/live.py:231: UserWarning: install "ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

┌──────────────────────────┬─────────┐
│ Total Estimated Cost     │ $5.1994 │
│ Number of Examples       │ 2000    │
│ Average cost per example │ $0.0026 │
└──────────────────────────┴─────────┘

───────────────────────────────────────────────── Prompt Example ──────────────────────────────────────────────────

You are an expert at extracting Person, Organization, Location, and Miscellaneous entities from text. Your job is to extract named entities mentioned in text, and classify them into one of the following categories.
Categories:
Location
Organization
Person
Miscellaneous
 

You will return the answer in CSV format, with two columns seperated by the % character. First column is the extracted entity and second column is the category. Rows in the CSV are separated by new line character.

Some examples with their output answers are provided below:

Example: MUNICH , Germany 1996-12-06
Output:
MUNICH%Location
Germany%Location

Example: PARIS 1996-12-06
Output:
PARIS%Location

Example: PARIS 1996-12-06
Output:
PARIS%Location

Example: COPENHAGEN 1996-12-06
Output:
COPENHAGEN%Location

Example: MANTUA , Italy 1996-12-06
Output:
Italy%Location

Now I want you to label the following example:
Example: SOCCER - JAPAN GET LUCKY WIN , CHINA IN SURPRISE DEFEAT .
Output:



───────────────────────────────────────────────────────────────────────────────────────────────────────────────────

In [18]:
# now, do the actual labeling
ds = agent.run(ds, max_items=100)

/opt/homebrew/lib/python3.10/site-packages/rich/live.py:231: UserWarning: install "ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

2023-06-14 13:05:14 openai INFO: error_code=None error_message='That model is currently overloaded with other requests. You can retry your request, or contact us through our help center at help.openai.com if the error persists. (Please include the request ID ece6f934aafb02c25d314adcb4606d28 in your message.)' error_param=None error_type=server_error message='OpenAI API error received' stream_error=False
2023-06-14 13:05:14 langchain.chat_models.openai WARNING: Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised RateLimitError: That model is currently overloaded with other requests. You can retry your request, or contact us through our help center at help.openai.com if the error persists. (Please include the request ID ece6f934aafb02c25d314adcb4606d28 in your message.).


Actual Cost: 0.0612


┏━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ f1     ┃ support ┃ threshold ┃ accuracy ┃ completion_rate ┃
┡━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ 0.8662 │ 100     │ -inf      │ 0.8894   │ 1.0             │
└────────┴─────────┴───────────┴──────────┴─────────────────┘

Total number of failures: 0

We are at 88.9% accuracy when labeling the first 100 examples. Let's see if we can use confidence scores to improve accuracy further by removing the less confident examples from our labeled set.

### Compute confidence scores


In [21]:
# Start computing confidence scores (using Refuel's LLMs)
os.environ["REFUEL_API_KEY"] = "xxxxxxxxxxxxxxxxx"

In [22]:
# set `compute_confidence` -> True
config["model"]["compute_confidence"] = True

In [23]:
agent = LabelingAgent(config=config)

In [24]:
from autolabel import AutolabelDataset

ds = AutolabelDataset("data/conll2003/test.csv", config=config)
agent.plan(ds)

/opt/homebrew/lib/python3.10/site-packages/rich/live.py:231: UserWarning: install "ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

┌──────────────────────────┬─────────┐
│ Total Estimated Cost     │ $5.1993 │
│ Number of Examples       │ 2000    │
│ Average cost per example │ $0.0026 │
└──────────────────────────┴─────────┘

───────────────────────────────────────────────── Prompt Example ──────────────────────────────────────────────────

You are an expert at extracting Person, Organization, Location, and Miscellaneous entities from text. Your job is to extract named entities mentioned in text, and classify them into one of the following categories.
Categories:
Location
Organization
Person
Miscellaneous
 

You will return the answer in CSV format, with two columns seperated by the % character. First column is the extracted entity and second column is the category. Rows in the CSV are separated by new line character.

Some examples with their output answers are provided below:

Example: MUNICH , Germany 1996-12-06
Output:
MUNICH%Location
Germany%Location

Example: PARIS 1996-12-06
Output:
PARIS%Location

Example: PARIS 1996-12-06
Output:
PARIS%Location

Example: COPENHAGEN 1996-12-06
Output:
COPENHAGEN%Location

Example: MANTUA , Italy 1996-12-06
Output:
Italy%Location

Now I want you to label the following example:
Example: SOCCER - JAPAN GET LUCKY WIN , CHINA IN SURPRISE DEFEAT .
Output:



───────────────────────────────────────────────────────────────────────────────────────────────────────────────────

In [25]:
ds = agent.run(ds, max_items=500)

/opt/homebrew/lib/python3.10/site-packages/rich/live.py:231: UserWarning: install "ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

/opt/homebrew/lib/python3.10/site-packages/rich/live.py:231: UserWarning: install "ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

2023-06-14 13:18:26 openai INFO: error_code=None error_message='That model is currently overloaded with other requests. You can retry your request, or contact us through our help center at help.openai.com if the error persists. (Please include the request ID eb687bb31abbab896586cabd17edb22c in your message.)' error_param=None error_type=server_error message='OpenAI API error received' stream_error=False
2023-06-14 13:18:26 langchain.chat_models.openai WARNING: Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised RateLimitError: That model is currently overloaded with other requests. You can retry your request, or contact us through our help center at help.openai.com if the error persists. (Please include the request ID eb687bb31abbab896586cabd17edb22c in your message.).


/opt/homebrew/lib/python3.10/site-packages/rich/live.py:231: UserWarning: install "ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

/opt/homebrew/lib/python3.10/site-packages/rich/live.py:231: UserWarning: install "ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

...

/opt/homebrew/lib/python3.10/site-packages/rich/live.py:231: UserWarning: install "ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

/opt/homebrew/lib/python3.10/site-packages/rich/live.py:231: UserWarning: install "ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

Metric: auroc: 0.7213
Actual Cost: 0.2271


┏━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ f1     ┃ support ┃ threshold ┃ accuracy ┃ completion_rate ┃
┡━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ 0.8321 │ 500     │ -inf      │ 0.7864   │ 1.0             │
│ 1.0    │ 2       │ 0.9937    │ 1.0      │ 0.004           │
│ 1.0    │ 3       │ 0.9929    │ 1.0      │ 0.006           │
│ 1.0    │ 6       │ 0.9914    │ 1.0      │ 0.012           │
│ 1.0    │ 8       │ 0.9876    │ 1.0      │ 0.016           │
│ 1.0    │ 10      │ 0.9809    │ 1.0      │ 0.02            │
│ 1.0    │ 11      │ 0.9778    │ 1.0      │ 0.022           │
│ 1.0    │ 12      │ 0.9776    │ 0.9677   │ 0.024           │
│ 1.0    │ 13      │ 0.9738    │ 0.9688   │ 0.026           │
│ 1.0    │ 14      │ 0.9701    │ 0.9744   │ 0.028           │
│ 1.0    │ 15      │ 0.9668    │ 0.975    │ 0.03            │
│ 1.0    │ 16      │ 0.9665    │ 0.9811   │ 0.032           │
│ 1.0    │ 17      │ 0.9663    │ 0.9818   │ 0.034           │
│ 1.0    │ 18      │ 0.964     │ 0.9828   │ 0.036           │
│ 1.0    │ 19      │ 0.9638    │ 0.9661   │ 0.038           │
│ 1.0    │ 20      │ 0.9615    │ 0.9722   │ 0.04            │
│ 1.0    │ 21      │ 0.9613    │ 0.9726   │ 0.042           │
│ 1.0    │ 22      │ 0.9598    │ 0.974    │ 0.044           │
│ 1.0    │ 23      │ 0.9583    │ 0.978    │ 0.046           │
│ 1.0    │ 24      │ 0.9574    │ 0.9813   │ 0.048           │
│ 1.0    │ 25      │ 0.9553    │ 0.9835   │ 0.05            │
│ 1.0    │ 26      │ 0.9534    │ 0.9844   │ 0.052           │
│ 1.0    │ 28      │ 0.9497    │ 0.9853   │ 0.056           │
│ 1.0    │ 29      │ 0.9487    │ 0.9781   │ 0.058           │
│ 1.0    │ 30      │ 0.9463    │ 0.9783   │ 0.06            │
│ 1.0    │ 31      │ 0.9449    │ 0.9786   │ 0.062           │
│ 1.0    │ 32      │ 0.9437    │ 0.9787   │ 0.064           │
│ 1.0    │ 34      │ 0.9421    │ 0.9793   │ 0.068           │
│ 1.0    │ 35      │ 0.9418    │ 0.9795   │ 0.07            │
│ 1.0    │ 36      │ 0.941     │ 0.9799   │ 0.072           │
│ 1.0    │ 37      │ 0.94      │ 0.9804   │ 0.074           │
│ 1.0    │ 38      │ 0.9383    │ 0.974    │ 0.076           │
│ 1.0    │ 39      │ 0.9375    │ 0.9748   │ 0.078           │
│ 1.0    │ 40      │ 0.9361    │ 0.975    │ 0.08            │
│ 1.0    │ 41      │ 0.9358    │ 0.9755   │ 0.082           │
│ 1.0    │ 42      │ 0.9353    │ 0.9758   │ 0.084           │
│ 1.0    │ 43      │ 0.9342    │ 0.9762   │ 0.086           │
│ 1.0    │ 44      │ 0.9331    │ 0.9765   │ 0.088           │
│ 1.0    │ 45      │ 0.9331    │ 0.9771   │ 0.09            │
│ 1.0    │ 46      │ 0.9319    │ 0.9775   │ 0.092           │
│ 1.0    │ 47      │ 0.9314    │ 0.9721   │ 0.094           │
│ 1.0    │ 48      │ 0.9301    │ 0.9724   │ 0.096           │
│ 1.0    │ 49      │ 0.9258    │ 0.973    │ 0.098           │
│ 1.0    │ 50      │ 0.9218    │ 0.974    │ 0.1             │
│ 1.0    │ 52      │ 0.9207    │ 0.9695   │ 0.104           │
│ 1.0    │ 53      │ 0.9184    │ 0.9697   │ 0.106           │
│ 1.0    │ 54      │ 0.9179    │ 0.9703   │ 0.108           │
│ 0.9825 │ 55      │ 0.9178    │ 0.9399   │ 0.11            │
│ 0.9833 │ 56      │ 0.9172    │ 0.9383   │ 0.112           │
│ 0.9792 │ 57      │ 0.9161    │ 0.9344   │ 0.114           │
│ 0.9793 │ 58      │ 0.9148    │ 0.935    │ 0.116           │
│ 0.9798 │ 64      │ 0.9147    │ 0.9365   │ 0.128           │
│ 0.9785 │ 65      │ 0.9136    │ 0.9346   │ 0.13            │
│ 0.9788 │ 66      │ 0.9133    │ 0.9318   │ 0.132           │
│ 0.9789 │ 67      │ 0.9131    │ 0.9321   │ 0.134           │
│ 0.979  │ 68      │ 0.9123    │ 0.9286   │ 0.136           │
│ 0.9791 │ 69      │ 0.9106    │ 0.9291   │ 0.138           │
│ 0.9796 │ 71      │ 0.909     │ 0.9307   │ 0.142           │
│ 0.9797 │ 72      │ 0.9083    │ 0.9312   │ 0.144           │
│ 0.9802 │ 74      │ 0.9074    │ 0.9326   │ 0.148           │
│ 0.9803 │ 75      │ 0.9057    │ 0.9293   │ 0.15            │
│ 0.9804 │ 76      │ 0.9056    │

Total number of failures: 0

Looking at the table above, we can see that if we set the confidence threshold at `0.7573`, we are able to label at 82% accuracy and getting a completion rate of 74%. This means, we would ignore all the data points where confidence score is less than `0.7656` (which would end up being around 26% of all samples). This would, however, guarantee a very high quality labeled dataset for us. 

In [26]:
ds = agent.run(ds, max_items=100)

2023-06-14 13:52:09 langchain.embeddings.openai WARNING: Retrying langchain.embeddings.openai.embed_with_retry.<locals>._embed_with_retry in 4.0 seconds as it raised APIConnectionError: Error communicating with OpenAI: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response')).


/opt/homebrew/lib/python3.10/site-packages/rich/live.py:231: UserWarning: install "ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

/opt/homebrew/lib/python3.10/site-packages/rich/live.py:231: UserWarning: install "ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

/opt/homebrew/lib/python3.10/site-packages/rich/live.py:231: UserWarning: install "ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

Metric: auroc: 0.6973
Actual Cost: 0.0027


┏━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ f1     ┃ support ┃ threshold ┃ accuracy ┃ completion_rate ┃
┡━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ 0.8705 │ 100     │ -inf      │ 0.894    │ 1.0             │
│ 1.0    │ 1       │ 0.9876    │ 1.0      │ 0.01            │
│ 1.0    │ 2       │ 0.9821    │ 1.0      │ 0.02            │
│ 1.0    │ 3       │ 0.9778    │ 1.0      │ 0.03            │
│ 1.0    │ 4       │ 0.9686    │ 1.0      │ 0.04            │
│ 1.0    │ 5       │ 0.965     │ 1.0      │ 0.05            │
│ 1.0    │ 6       │ 0.9635    │ 1.0      │ 0.06            │
│ 1.0    │ 8       │ 0.9542    │ 1.0      │ 0.08            │
│ 1.0    │ 9       │ 0.9437    │ 1.0      │ 0.09            │
│ 1.0    │ 12      │ 0.9421    │ 1.0      │ 0.12            │
│ 1.0    │ 13      │ 0.9281    │ 1.0      │ 0.13            │
│ 1.0    │ 14      │ 0.9257    │ 1.0      │ 0.14            │
│ 1.0    │ 15      │ 0.9138    │ 1.0      │ 0.15            │
│ 1.0    │ 16      │ 0.9133    │ 0.9872   │ 0.16            │
│ 1.0    │ 17      │ 0.9127    │ 0.9873   │ 0.17            │
│ 1.0    │ 18      │ 0.9056    │ 0.9877   │ 0.18            │
│ 1.0    │ 19      │ 0.9029    │ 0.9878   │ 0.19            │
│ 0.9942 │ 20      │ 0.8955    │ 0.9651   │ 0.2             │
│ 0.9944 │ 22      │ 0.8922    │ 0.9667   │ 0.22            │
│ 0.9783 │ 23      │ 0.892     │ 0.9362   │ 0.23            │
│ 0.9789 │ 24      │ 0.8895    │ 0.9381   │ 0.24            │
│ 0.9792 │ 25      │ 0.8892    │ 0.9388   │ 0.25            │
│ 0.9796 │ 26      │ 0.8879    │ 0.94     │ 0.26            │
│ 0.9804 │ 28      │ 0.8863    │ 0.9423   │ 0.28            │
│ 0.9697 │ 35      │ 0.8676    │ 0.9483   │ 0.35            │
│ 0.9633 │ 37      │ 0.8639    │ 0.9355   │ 0.37            │
│ 0.9639 │ 38      │ 0.8583    │ 0.9365   │ 0.38            │
│ 0.9603 │ 39      │ 0.8575    │ 0.9297   │ 0.39            │
│ 0.9609 │ 40      │ 0.8557    │ 0.9308   │ 0.4             │
│ 0.9612 │ 41      │ 0.8492    │ 0.9313   │ 0.41            │
│ 0.9624 │ 43      │ 0.8478    │ 0.9333   │ 0.43            │
│ 0.963  │ 45      │ 0.8362    │ 0.9343   │ 0.45            │
│ 0.9603 │ 46      │ 0.8339    │ 0.9357   │ 0.46            │
│ 0.9606 │ 47      │ 0.8338    │ 0.9362   │ 0.47            │
│ 0.9583 │ 49      │ 0.8297    │ 0.9379   │ 0.49            │
│ 0.9589 │ 50      │ 0.8283    │ 0.932    │ 0.5             │
│ 0.9562 │ 51      │ 0.8261    │ 0.9329   │ 0.51            │
│ 0.9508 │ 53      │ 0.8232    │ 0.9216   │ 0.53            │
│ 0.9515 │ 55      │ 0.8203    │ 0.9226   │ 0.55            │
│ 0.949  │ 56      │ 0.8148    │ 0.9172   │ 0.56            │
│ 0.9494 │ 57      │ 0.8123    │ 0.9177   │ 0.57            │
│ 0.947  │ 58      │ 0.8081    │ 0.9187   │ 0.58            │
│ 0.9325 │ 59      │ 0.8077    │ 0.9074   │ 0.59            │
│ 0.9305 │ 60      │ 0.8055    │ 0.9085   │ 0.6             │
│ 0.9309 │ 61      │ 0.8021    │ 0.9091   │ 0.61            │
│ 0.929  │ 62      │ 0.7997    │ 0.9102   │ 0.62            │
│ 0.9271 │ 63      │ 0.7989    │ 0.9059   │ 0.63            │
│ 0.9195 │ 64      │ 0.7921    │ 0.9064   │ 0.64            │
│ 0.9183 │ 66      │ 0.7837    │ 0.908    │ 0.66            │
│ 0.9086 │ 70      │ 0.7687    │ 0.9111   │ 0.7             │
│ 0.9091 │ 71      │ 0.7669    │ 0.9061   │ 0.71            │
│ 0.9062 │ 73      │ 0.7576    │ 0.9081   │ 0.73            │
│ 0.9082 │ 74      │ 0.7551    │ 0.8989   │ 0.74            │
│ 0.9051 │ 78      │ 0.7368    │ 0.9031   │ 0.78            │
│ 0.9034 │ 79      │ 0.7358    │ 0.9036   │ 0.79            │
│ 0.9021 │ 80      │ 0.7314    │ 0.9045   │ 0.8             │
│ 0.9014 │ 81      │ 0.7312    │ 0.901    │ 0.81            │
│ 0.8959 │ 85      │ 0.7172    │ 0.9038   │ 0.85            │
│ 0.8964 │ 86      │ 0.7152    │ 0.9043   │ 0.86            │
│ 0.8884 │ 87      │ 0.7099    │ 0.9      │ 0.87            │
│ 0.8879 │ 88      │ 0.7098    │ 0.9014   │ 0.88            │
│ 0.8884 │ 89      │ 0.6882    │

Total number of failures: 0

Looking at the table above, we can see that if we set the confidence threshold at `0.7314`, we are able to label at 90.45% accuracy and getting a completion rate of 80%. This means, we would ignore all the data points where confidence score is less than `0.7314` (which would end up being around 20% of all samples). This would, however, guarantee a very high quality labeled dataset for us. 